<a href="https://colab.research.google.com/github/Bao3333/iKnowFish/blob/main/Rich_map1003.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###基本步驟

In [ ]:
# 連接Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 安裝套件
!pip install line-bot-sdk flask flask-ngrok pyngrok

In [ ]:
# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)
# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

In [ ]:
# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

# 生成實體物件
line_bot_api = LineBotApi("Mi4JsyDs+dGNuZPFlm7SqmlM2YTE0xxDkm8vkRolkSFrA9E2JQQfQKU1hUfayQhk1Po5iOrCCwV3XdwWPxKQH/JwGHxNG4y4qjjc7iLItABk0uTW+KSB/GyHKDoeQyrIE7zzw0jodQpP6Wm3R1BrFgdB04t89/1O/w1cDnyilFU=")
handler = WebhookHandler("8259df02f27ce24a385057e93d26655f")

In [ ]:
# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body:" + body)
    print(body)

    # 記錄用戶log
    f = open("/content/drive/MyDrive/ai-event.log", "a")
    f.write(body)
    f.close()

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

###Rich Menu 設置

In [ ]:
# 載入linebot.models
from linebot.models import *
import requests

#設定Rich Menu大小、位置及功能
rich_menu = RichMenu(
    size=RichMenuSize(width=2500, height=833),
    selected=False,
    name="Nice richmenu",
    chat_bar_text="Tap here",
    areas =[
        RichMenuArea(
            bounds=RichMenuBounds(x=0, y=0, width=833, height=554),
            action=MessageAction(label="情境互動", text="開啟情境互動任務")),
        RichMenuArea(
            bounds=RichMenuBounds(x=834, y=0, width=833, height=554),
            action=URIAction(label='魚類辨識', uri='https://line.me/R/nv/camera/')),
        RichMenuArea(
            bounds=RichMenuBounds(x=1667, y=0, width=833, height=554),
            action=URIAction(
                label='打開地圖', 
                uri='https://www.nmmst.gov.tw/chhtml/content/312')),
        RichMenuArea(
            bounds=RichMenuBounds(x=0, y=554, width=2500, height=277),
            action=PostbackAction(
                label='其他資訊',
                data="rich_menu_others"
            ))
        ]
)

In [ ]:
# 取得該Rich Menu的id

rich_menu_default_id = line_bot_api.create_rich_menu(rich_menu=rich_menu)
print(rich_menu_default_id)

richmenu-63ee00725089c0eeffb4ab53b947d099


In [ ]:
# 將特定的Rich Menu(用id來指定要哪個)與想要的圖片綁在一起

with open("/content/drive/MyDrive/Tibame_ai工程師班/期末專題/LineBot/Rich_Menu.jpg", 'rb') as f:
    line_bot_api.set_rich_menu_image(rich_menu_default_id, "image/jpeg", f)

In [ ]:
# 將綁好的rich Menu設定為預設圖文選單(不針對使用者id, 所有人都用的)

line_bot_api.set_default_rich_menu(rich_menu_default_id)

###Follow Event-記錄用戶資訊、發送歡迎訊息

In [ ]:
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
     # 將用戶資訊存在檔案內
    with open("/content/drive/MyDrive/users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\n')

    #line_bot_api.link_rich_menu_to_user(event.source.user_id, rich_menu_id)

    # 回覆文字消息與圖片消息
    line_bot_api.reply_message(
        event.reply_token,
        [TextSendMessage('非常感謝您的關注!')]
    )

###Rich Menu-情境互動連接(暫定Button Template形式進行)

In [ ]:
# 建立Button Template

story_button1 = TemplateSendMessage(
    alt_text="Buttons template",
    template=ButtonsTemplate(title="歡迎來到海科館, 接下來請依照提示完成任務, 完成後將能夠獲得神秘小禮物",
                             text="準備好要開啟海科館大冒險了嗎",
                             thumbnail_image_url="https://i.imgur.com/ECtbnK4.jpg",
                             actions=[
                                 {"type":"message",
                                  "label":"我準備好了",
                                  "text":"我準備好了",
                                 },
                                 {"type":"message",
                                  "label":"小等一下",
                                  "text":"小等一下",
                                 }
                                      ]
                             )
                                    )

In [ ]:
# 建立觸發回覆

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
  if event.message.text == "開啟情境互動任務":
    line_bot_api.reply_message(event.reply_token, 
                               story_button1)


###Rich Menu-其他功能Quickreply Button建立

In [ ]:
@handler.add(PostbackEvent)
def handle_postback_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
    if event.postback.data == 'rich_menu_others':
        flex_message = TextSendMessage(
            text='最新活動及魚種介紹',
            quick_reply=QuickReply(
                items=[
                QuickReplyButton(action=MessageAction(label="其他活動", text="其他活動")),
                QuickReplyButton(action=MessageAction(label="魚種介紹", text="魚種介紹")),
                QuickReplyButton(action=URIAction(
                    label='進入官網', 
                    uri='https://www.nmmst.gov.tw/chhtml/'
                ))]
            )
        )
        line_bot_api.reply_message(
            event.reply_token,
            [
                flex_message
            ]
        )

In [ ]:
app.run()